In [1]:
from EmbeddingDatabase import EmbeddingDatabase
from EmbeddingGenerator import EmbeddingGenerator
from RelevanceEvaluator import RelevanceEvaluator
from TextProcessor import TextProcessor
from langchain.chat_models.gigachat import GigaChat
from tqdm import tqdm
import os 

s:\LLM\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = 'data/Koroleva_Klinkov.txt' # путь до данных 
save_to_path_file = 'data/Koroleva_Klinkov_embeddings.pkl' # путь сохранения ембейдингов
top_k = 5 # топ релевантных запросов 
chunk_size = 400 # размер чанка при разбиении текста

query = '''
Зерги присутствуют на планете, вне всяких сомнений. Они уже победили Дюка и выдворили его обратно на 
орбиту Чара.
'''
# параметры для запуска llm
llm_credentials = "u api key"
llm_scope = "GIGACHAT_API_PERS"
llm_model = "GigaChat"

In [3]:
# 1. Обработка текста
text_processor = TextProcessor(file_path)
data_chunks = text_processor.get_chunks(chunk_size=chunk_size)

In [4]:
# 2. Генерация эмбеддингов
embedding_generator = EmbeddingGenerator()
db = EmbeddingDatabase()

if os.path.exists(save_to_path_file):
    # 3. Работа с базой данных эмбеддингов
    db.load_from_file(save_to_path_file)
else: 
    db_embeddings = [embedding_generator.get_embedding(chunk) for chunk in tqdm(data_chunks)]
    # 3. Работа с базой данных эмбеддингов
    db.add_embeddings(db_embeddings, data_chunks)
    db.save_to_file(save_to_path_file)

s:\LLM\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
s:\LLM\.venv\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this m

In [5]:
# 4. Оценка релевантности
question_embedding = embedding_generator.get_embedding(query)
top_k_indices = db.find_top_k(question_embedding, k=top_k)

In [6]:
# 5. Оценка релевантности llm
giga = GigaChat(
    credentials=llm_credentials , 
    scope=llm_scope , 
    model=llm_model , 
    verify_ssl_certs=False)

evaluator = RelevanceEvaluator(giga)

relevant_chunks = evaluator.determine_relevance(query, data_chunks, top_k_indices)

# 6. Подсчет метрик
evaluator.evaluate_metrics(relevant_chunks, k=top_k)

s:\LLM\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Average Precision at 5: 0.8666666666666667
Precision at 5: 0.6
